# Objetivo

Aplicar o K Nearest Neighbors Na base toda. 

# Método

A partir do conjunto reduzido de features (39) 100k rows, aplicar:
- fillna <b> OK </b>

- LabelEncoder (ou algo do tipo) <b> 'parcial' </b> - Travou 
- KNN

# Leitura dos arquivos

In [1]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## Leitura da lista de features a excluir

# Carregamento Clientes

## Redução no número de features
Só rodar caso seja necessário filtrar mais vezes

In [2]:
p1 = pd.read_csv('estaticos_market.csv')
p1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 462298 entries, 0 to 462297
Columns: 182 entries, Unnamed: 0 to qt_filiais
dtypes: bool(9), float64(144), int64(2), object(27)
memory usage: 614.1+ MB


In [3]:
exclude_df = pd.read_csv('exclude_variables_generated_from_profiling.csv')
exclude_variables = exclude_df.values.squeeze().tolist()
len(exclude_variables)


143

In [5]:
p1.columns

Index(['Unnamed: 0', 'id', 'fl_matriz', 'de_natureza_juridica', 'sg_uf',
       'natureza_juridica_macro', 'de_ramo', 'setor', 'idade_empresa_anos',
       'idade_emp_cat',
       ...
       'media_meses_servicos', 'max_meses_servicos', 'min_meses_servicos',
       'qt_funcionarios', 'qt_funcionarios_12meses', 'qt_funcionarios_24meses',
       'tx_crescimento_12meses', 'tx_crescimento_24meses', 'tx_rotatividade',
       'qt_filiais'],
      dtype='object', length=182)

In [6]:
p1.drop(columns=exclude_variables, inplace= True)
p1.drop(columns=['Unnamed: 0'], inplace= True)
p1.set_index('id',inplace=True)
p1.shape

(462298, 37)

In [ ]:
#p1.to_csv('estaticos_49_features.csv')

## Carregamento de clientes já filtrados


In [ ]:
p1 = pd.read_csv('estaticos_41_features.csv')#,nrows = 200000)

In [ ]:
p1.columns

In [ ]:
p1.drop(columns=['Unnamed: 0'], inplace= True)
p1.set_index('id',inplace=True)
p1.shape

# Managing NaNs


Para usar nos dados do portfólio:
- passar as strings para valor numérico
- passar bool para int
- fazer fillna para todos
- - em categoricas -> other
- - em numericas -> 0

Estamos usando o LabelEncoder

In [7]:
fill_dict = p1.dtypes.to_dict()
cat2bool = ['fl_optante_simei','fl_optante_simples','fl_passivel_iss']

for feature in fill_dict:
    if(fill_dict[feature] == bool):
        fill_dict[feature] = False
    elif(fill_dict[feature] == object):
        fill_dict[feature] = 'other'
    else:
        fill_dict[feature] = 0
    if feature in cat2bool:
        fill_dict[feature] = False
        
fill_dict


{'fl_matriz': False,
 'de_natureza_juridica': 'other',
 'sg_uf': 'other',
 'natureza_juridica_macro': 'other',
 'de_ramo': 'other',
 'setor': 'other',
 'idade_empresa_anos': 0,
 'idade_emp_cat': 'other',
 'fl_me': False,
 'fl_sa': False,
 'fl_mei': False,
 'fl_ltda': False,
 'dt_situacao': 'other',
 'fl_st_especial': False,
 'fl_email': False,
 'fl_telefone': False,
 'fl_rm': 'other',
 'nm_divisao': 'other',
 'nm_segmento': 'other',
 'fl_optante_simples': False,
 'fl_optante_simei': False,
 'sg_uf_matriz': 'other',
 'de_saude_tributaria': 'other',
 'de_saude_rescencia': 'other',
 'nu_meses_rescencia': 0,
 'de_nivel_atividade': 'other',
 'empsetorcensitariofaixarendapopulacao': 0,
 'nm_meso_regiao': 'other',
 'nm_micro_regiao': 'other',
 'fl_passivel_iss': False,
 'qt_socios': 0,
 'idade_media_socios': 0,
 'qt_socios_feminino': 0,
 'de_faixa_faturamento_estimado': 'other',
 'de_faixa_faturamento_estimado_grupo': 'other',
 'vl_faturamento_estimado_aux': 0,
 'vl_faturamento_estimado_grupo

In [8]:
p1.fillna(value=fill_dict, inplace=True)
p1[cat2bool] = p1[cat2bool].astype(bool)

bool2numeric = p1.columns[p1.dtypes == bool]
p1[bool2numeric] = p1[bool2numeric].astype(int)

## Encoding

In [9]:
cat = p1.columns[p1.dtypes == object]
cat
not_cat = p1.columns[p1.dtypes != object]

In [10]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()
#NÃO É MAIS NECESSÁRIO
#p1_scaled = encoder.fit_transform(p1)

In [11]:

# Usando LabelEncoder
#https://chrisalbon.com/machine_learning/preprocessing_structured_data/convert_pandas_categorical_column_into_integers_for_scikit-learn/
from sklearn import preprocessing
# Create a label (category) encoder object
le = preprocessing.LabelEncoder()

In [12]:
p1_le = pd.DataFrame()
p1_le[cat] = p1[cat].apply(lambda col: le.fit_transform(col))


In [13]:
p1_encoded = pd.concat([p1[not_cat],p1_le], axis =1)

In [14]:
p1_encoded.dtypes

fl_matriz                                  int64
idade_empresa_anos                       float64
fl_me                                      int64
fl_sa                                      int64
fl_mei                                     int64
fl_ltda                                    int64
fl_st_especial                             int64
fl_email                                   int64
fl_telefone                                int64
fl_optante_simples                         int64
fl_optante_simei                           int64
nu_meses_rescencia                       float64
empsetorcensitariofaixarendapopulacao    float64
fl_passivel_iss                            int64
qt_socios                                float64
idade_media_socios                       float64
qt_socios_feminino                       float64
vl_faturamento_estimado_aux              float64
vl_faturamento_estimado_grupo_aux        float64
de_natureza_juridica                       int64
sg_uf               

# PCA


Aplicar o PCA para redução de features

In [ ]:
p1_encoded.iloc[:,:35]

In [16]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
stdScaler = MinMaxScaler()

p1_scaled = stdScaler.fit_transform(p1_encoded.iloc[:,:35])

pca = PCA().fit(p1_encoded)
#pca.fit_transform(p1_encoded)

In [17]:
evr = pca.explained_variance_ratio_
ev = pca.explained_variance_

evr

array([9.99981021e-01, 1.89792375e-05, 5.78359568e-14, 2.71481537e-14,
       1.23322077e-17, 1.08135143e-17, 1.04641177e-17, 6.53662612e-18,
       2.40660499e-18, 1.95879224e-18, 9.97701877e-19, 6.22042971e-19,
       5.75028299e-19, 4.46531345e-19, 2.98995316e-19, 6.47766828e-20,
       5.56686294e-20, 4.34062198e-20, 1.94131610e-20, 1.66924925e-20,
       1.53475544e-20, 1.09781490e-20, 9.43597300e-21, 6.64857296e-21,
       4.94345796e-21, 4.33416669e-21, 3.65916596e-21, 2.83448417e-21,
       2.35533577e-21, 2.14674744e-21, 1.62403125e-21, 6.28619074e-22,
       3.68900982e-22, 3.40065559e-22, 7.00489560e-23, 4.03215734e-23,
       3.08366323e-24])

In [18]:
pca.components_.shape

(37, 37)

In [ ]:
g = sns.lineplot(np.arange(len(evr)), np.cumsum(evr))
g.axes.axhline(0.95, ls="--", color="red")
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance');

In [25]:
from sklearn.neighbors import NearestNeighbors

nearest = NearestNeighbors()

nearest.fit(p1_sample)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [43]:
p1_encoded.head(1)


,fl_matriz,idade_empresa_anos,fl_me,fl_sa,fl_mei,fl_ltda,fl_st_especial,fl_email,fl_telefone,fl_optante_simples,...,nm_divisao,nm_segmento,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,de_nivel_atividade,nm_meso_regiao,nm_micro_regiao,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo
id,,,,,,,,,,,,,,,,,,,,,
a6984c3ae395090e3bee8ad63c3758b110de096d5d819583a784a113726db849,1,14.457534,0,0,0,0,0,1,1,1,...,32,10,19,4,0,0,6,48,2,2


In [45]:
nearest.kneighbors(p1_encoded.tail(1))

(array([[21.37755833, 31.89044038, 35.18529628, 36.72873534, 38.36665219]]),
 array([[5601,  962, 9169,  880, 7029]]))

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(p1_encoded, pd.Series(np.zeros(len(p1_encoded))), 
                     train_size=0.02, random_state=17) 
from sklearn.cluster import AgglomerativeClustering
AC = AgglomerativeClustering(n_clusters=25, linkage='ward')
AC.fit(X_train)
labels = AC.labels_


In [19]:
p1_sample = p1_encoded.sample(n=10000)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KN = KNeighborsClassifier(n_neighbors=3)
KN.fit(X_train,labels)
clusters_fulldata = KN.predict(p1_scaled)
p1_scaled['cluster'] = pd.Series(clusters_fulldata)

In [ ]:
p1_scaled['cluster'].value_counts()

## Conclusão até o momento

In [ ]:
- NaNs limpos
- 